# Problem Statement
**beatit.ai** is one of the a music streaming startup in South Asia. 
They offer their services to millions of people, supported by advertising and paid subscriptions. It uses free or discounted trials to entice a customer who arrives on their platform. However, with the arrival of some new competitors the company’s churn rate is rising high. 

The task at hand is to predict the propensity of customer churn for the company. In the project, we would like to build a model which can predict in advance the customers who are at risk to cancel the beatit.ai music streaming service based on available data which is the user's past activity and interaction logs with the service. 

This will assist in identifying the probability of customer churn in the future so that preventive action can be taken proactively. 



# 1. Data Preparation
The first major task is to access and evaluate the data. The Data, to be used is coming from several sources and contains information about each user's subscription and streaming activities.

1.   User Profile data: **members.csv**. This data includes the user persona like user’s age, city and their registration time.
2.   User Logs data: **user_logs.csv**. This data consist of each user's listening behaviour in terms of their songs played in each day
3.   User Transaction data:  **transations.csv**. This data consist of details like payment method or whether the subscription was cancelled.
4.   Historical data: **train.csv** . This data consists of user IDs and whether these users have churned or not.


In [ ]:
# function
import os
import sys

new_directory = "E:/airflow/airflow"
current_directory = os.getcwd()
scripts_path = os.path.abspath(os.path.join(os.getcwd(), '../scripts'))

def change_directory(current_directory, new_directory,scripts_path):
    # Get the current working directory
    print(f'Current directory: {current_directory}')
    # Define the path to change to
    #new_directory = "E:/airflow/airflow"
    try:
        # Change the current working directory
        os.chdir(new_directory)
        # Verify the change
        current_directory = os.getcwd()
        print(f'Current directory changed to: {current_directory}')
    except FileNotFoundError:
        print(f'Error: The directory "{new_directory}" does not exist.')
    except PermissionError:
        print(f'Error: Permission denied to change to "{new_directory}".')
    except Exception as e:
        print(f'An unexpected error occurred: {e}')
    # Add the scripts directory to the Python path
    sys.path.append(scripts_path)

    
change_directory(current_directory, new_directory, scripts_path)

In [ ]:
# Setting up all directory
root_folder = new_directory
database_path = root_folder+"/database/"
data_directory = root_folder+"/data/raw/"
data_profile_path = root_folder+"/data/profile_report/"
intermediate_data_path = root_folder+"/data/interim/"
final_processed_data_path = root_folder+"/data/processed/"

old_data_directory = root_folder+"/data/raw/"
new_data_directory = root_folder+"/data/new/"
intermediate_path = root_folder+"/data/interim/"

# Database
db_path = root_folder+"/database/"
db_file_name = "feature_store_v01.db"
drfit_db_name = "drift_db_name.db"
date_columns = ['registration_init_time','transaction_date_min','transaction_date_max','membership_expire_date_max','last_login']


### 1.1 Importing packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline
from scripts.utils import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime
from pandas_profiling import ProfileReport

In [ ]:
t = time.time()

 ### 1.2 Reading Data

In [ ]:
#this function is also available in utils.py

# def load_data(file_path_list):
#     data = []
#     for eachfile in file_path_list:
#         data.append(pd.read_csv(eachfile))
#     return data

In [ ]:
%%time
members, user_logs, transactions, train  = load_data( [
                                                            f"{data_directory}members_profile.csv",
                                                            f"{data_directory}userlogs.csv",
                                                            f"{data_directory}transactions_logs.csv",
                                                            f"{data_directory}churn_logs.csv"
                                                            ]
                                                          )

In [ ]:
print(members.shape)
print(transactions.shape)
print(user_logs.shape)
print(train.shape)

Since there are four data scources, it is important to understand the distribution of each data before joining them.

In [ ]:
members.head()

The data is structured as:
* msno
* city
* bd: age. Note: this column has outlier values ranging from -7000 to 2015, please use your judgement.
* gender
* registered_via: registration method
* registration_init_time: format %Y%m%d
* expiration_date: format %Y%m%d, taken as a snapshot at which the member.csv is extracted. Not representing the actual churn behavior.

 Note that not every user in the dataset is available.

In [ ]:
transactions.head()

The data is structured as:
* msno: user id
* payment_method_id: payment method
* payment_plan_days: length of membership plan in days
* plan_list_price: in New Taiwan Dollar (NTD)
* actual_amount_paid: in New Taiwan Dollar (NTD)
* is_auto_renew
* transaction_date: format %Y%m%d
* membership_expire_date: format %Y%m%d
* is_cancel: whether or not the user canceled the membership in this transaction.

In [ ]:
print('Total unique records: ',transactions.msno.nunique())

In [ ]:
transactions[transactions['msno']=="Qw6UVFUknPVOLxSSsejinxU/8a5/AgmiWMvPoEt0rik="]

In [ ]:
user_logs.head()

The data is structured as:
* msno: user id
* date: format %Y%m%d
* num_25: # of songs played less than 25% of the song length
* num_50: # of songs played between 25% to 50% of the song length
* num_75: # of songs played between 50% to 75% of of the song length
* num_985: # of songs played between 75% to 98.5% of the song length
* num_100: # of songs played over 98.5% of the song length
* num_unq: # of unique songs played
* total_secs: total seconds played

In [ ]:
print('Total unique records: ',user_logs.msno.nunique())

In [ ]:
train

Here, the column **msno** represents the unique identity of a service subscriber. 
The transaction data provides the information of each subscriber and their transactional details with respect to the service they have subscribed to. Similarly the data in the user logs table provides information about each user and their streaming behaviour on a daily level.

Here, the transaction and streaming logs are quite verbose with multiple records being recorded for a subscriber on a given date. On dates where there is no activity, no entries are found for a subscriber in these tables.

Additionaly, certain information in the above data have been masked for privacy concerns. 

> Definition of a **churned user**:
A subscriber is identified as a churned user if he/she fails to renew their subscription within 30 days after their current membership expires.
Upon inspecting the data, we can observe that a majority of beatit.ai's subscription length is 30 days, therefore it can be said that a lot of users re-subscribe every month.

 ### 1.2.1 Data conversion for memory reducton

In [ ]:
#this function is present in utils.py

# def compress_dataframes(list_of_dfs):
#     final_df = []
#     for eachdf in list_of_dfs:
#         original_size = (eachdf.memory_usage(index=True).sum())/ 1024**2
#         int_cols = list(eachdf.select_dtypes(include=['int']).columns)
#         float_cols = list(eachdf.select_dtypes(include=['float']).columns)
#         for col in int_cols:
#             if ((np.max(eachdf[col]) <= 127) and(np.min(eachdf[col] >= -128))):
#                 eachdf[col] = eachdf[col].astype(np.int8)
#             elif ((np.max(eachdf[col]) <= 32767) and(np.min(eachdf[col] >= -32768))):
#                 eachdf[col] = eachdf[col].astype(np.int16)
#             elif ((np.max(eachdf[col]) <= 2147483647) and(np.min(eachdf[col] >= -2147483648))):
#                 eachdf[col] = eachdf[col].astype(np.int32)
#             else:
#                 eachdf[col] = eachdf[col].astype(np.int64)
    
#         for col in float_cols:
#             eachdf[col] = eachdf[col].astype(np.float16)
#         compressed_size = (eachdf.memory_usage(index=True).sum())/ 1024**2
        
#         final_df.append((eachdf,original_size,compressed_size))
        
#     return final_df

In [ ]:
%%time
members_c, transactions_c, user_logs_c = compress_dataframes([members, transactions, user_logs])
members = members_c[0]
transactions = transactions_c[0]
user_logs = user_logs_c[0]

In [ ]:
print("members DF before compress was in MB ,",members_c[1], "and after compress , ", members_c[2])
print("transactions DF before compress was in MB ,",transactions_c[1], "and after compress , ", transactions_c[2])
print("user_logs DF before compress was in MB ,",user_logs_c[1], "and after compress , ", user_logs_c[2])

 ### 1.3 EDA

In [ ]:
#function available in utils.py

# def count_plot(dataframe, list_of_columns):
#     final_plot = []
#     for eachcol in list_of_columns:
#         plt.figure(figsize=(15,5))
#         unique_features = dataframe[eachcol].unique()
#         if dataframe[eachcol].dtype =='int64':
#             unique_features=sorted(unique_features)
#         sns.countplot(x=eachcol, data=dataframe , order = unique_features)
#         plt.xlabel(eachcol)
#         plt.ylabel('Count')
#         plt.title("Frequency plot of {} Count".format(eachcol))
#         plt.show()

In [ ]:
# %matplotlib inline   
#if plots don't come, uncomment and run above cell one more time 

In [ ]:
%%time
count_plot(members, ['city','gender','registered_via'])

**Observation:**

* There are total of 21 Cities Encoded ( there is no City "2" in the data set). 
* There are Class of "3", "4", "7", "9", "11","13" listed as registration method.  

Kindly note that there is additional "10", and "16" class of cities listed in Member Data set but there are missing when we merged the data set ( see below). 
*  There are almost equal percentage of Male and Female, but more than half of the data is missing in gender field. We have see how to fill the missing entries or label them as third category. 

> Details of Registration init time can't be explored without converting the column to datatime.

In [ ]:
#this function is also available in utils.py

# def fix_time_in_df(dataframe, column_name, expand=False):
#     if not expand:
#         dataframe[column_name] = dataframe[column_name].astype('str')
#         return pd.to_datetime(dataframe[column_name])
#     else:
#         dataframe_new = dataframe.copy()
#         dataframe_new[column_name] = dataframe_new[column_name].astype('str')
#         dataframe_new[column_name] = pd.to_datetime(dataframe_new[column_name])
#         #Extracting the date time year component
#         dataframe_new[f"{column_name}_year"] = pd.DatetimeIndex(dataframe_new[column_name]).year
#         #Extracting the date time year component
#         dataframe_new[f"{column_name}_month"] = pd.DatetimeIndex(dataframe_new[column_name]).month
#         #Extracting the date time year component
#         dataframe_new[f"{column_name}_day"] = pd.DatetimeIndex(dataframe_new[column_name]).day_name()
      
#         return dataframe_new
    

In [ ]:
%%time
members_new = fix_time_in_df(members, 'registration_init_time', expand=True)

In [ ]:
members_new.head()

In [ ]:
count_plot(members_new,['registration_init_time_year','registration_init_time_month','registration_init_time_day'])

**Observations:**

* Registration trend has increased yearly, though there was a dip in 2014. Due to data upto few months in 2017, there is a dip.
* Registration monthly trends are high in year end and year starting months. In between there is a smooth valley formation.
* Registration daily trends are high on weekends.

In [ ]:
%%time
members['registration_init_time'] = fix_time_in_df(members, 'registration_init_time', expand=False)

In [ ]:
members.head()

### 1.3.1 EDA using Pandas Profiling

In [ ]:
#this function is also available in utils.py
# import time

# def get_data_profile(dataframe,html_save_path, 
#                      embed_in_cell=True,take_sample=False, sample_frac=0.5, dataframe_name="data"):
#     if take_sample:
#         dataframe = dataframe.sample(frac=sample_frac)
#     if embed_in_cell:
#         profile = ProfileReport(dataframe, title=f"{dataframe_name} Data Summary Report")
#         return profile.to_notebook_iframe()
#     else:
#         profile = ProfileReport(dataframe, title=f"{dataframe_name} Data Summary Report")
#         timestamp = str(int(time.time()))
#         filename = f"{dataframe_name}_data_profile_{timestamp}"
#         profile.to_file(html_save_path+filename+".html")
#         return "Your Data Profile has been saved at .. ",html_save_path+filename+".html"

In [ ]:
%%time
#taking sample
get_data_profile(members,html_save_path=None, 
                     embed_in_cell=True,take_sample=True, sample_frac=0.01, 
                dataframe_name='Members')

**Observations**:


#Write your observations from the profile report created above



In [ ]:
%%time
get_data_profile(train,html_save_path=None, 
                     embed_in_cell=True,take_sample=False, sample_frac=0.01, 
                dataframe_name='Train')

In [ ]:
%%time
# Convert all float16 columns to float32
user_logs = user_logs.astype({col: 'float32' for col in user_logs.select_dtypes('float16').columns})

get_data_profile(user_logs,html_save_path=data_profile_path, 
                     embed_in_cell=False,take_sample=True, sample_frac=0.001, 
                dataframe_name='user_logs')

### 1.5 Feature Engineering

#### Registration Features

In [ ]:
#these functions are available in utils.py

# def get_data_describe(dataframe,round_num=2):
#     return round(dataframe.describe(),round_num)

# def get_data_na_values(dataframe, round_num=2):
#     return pd.DataFrame({'%missing_values':round(dataframe.isna().sum()/dataframe.shape[0],round_num)})

# def get_fill_na_dataframe(dataframe, column_name, value='mean'):
#     if value != 'mean' and value !='mode':
#         return dataframe[column_name].fillna(value)
#     elif value == 'mean':
#         value = dataframe[column_name].mean()
#         return dataframe[column_name].fillna(value)
#     elif value == 'mode':
#         value = dataframe[column_name].mode()
#         return dataframe[column_name].fillna(value)

# def get_convert_column_dtype(dataframe, column_name, data_type='str'):
#     if data_type == 'str':
#         return dataframe[column_name].astype('str')
#     elif data_type == 'int':
#         return dataframe[column_name].astype('int')
#     elif data_type == 'float':
#         return dataframe[column_name].astype('float')

In [ ]:
get_data_describe(members)

In [ ]:
get_data_na_values(members)

In [ ]:
#Replacing missing values in gender
members['gender'] = get_fill_na_dataframe(members, 'gender', value="others")

In [ ]:
members.info()

In [ ]:
members['registered_via'] = get_convert_column_dtype(members, 'registered_via', data_type='str')
members['city'] = get_convert_column_dtype(members, 'city', data_type='str')

In [ ]:
members.info()

In [ ]:
members.head()

#### Transactional features

The data here is present in one-many format, as one user can have multiple transaction samples. Therefore while joining the entire data, we need to convert the transactions data in one-one format.
But before that let's format the data and process it

In [ ]:
get_data_describe(transactions)

In [ ]:
#Missing Values
get_data_na_values(transactions)

In [ ]:
%%time
transactions['transaction_date'] = utils.fix_time_in_df(transactions, 'transaction_date', expand=False)
transactions['membership_expire_date'] = utils.fix_time_in_df(transactions, 'membership_expire_date', expand=False)

In [ ]:
transactions.head()

In [ ]:
#these functions are also available in utils.py

# def get_groupby(dataframe, by_column, agg_dict=None, agg_func = 'mean', simple_agg_flag=True, reset_index=True):
#     if reset_index:
#         if simple_agg_flag:
#             return dataframe.groupby(by_column).agg(agg_func).reset_index()
#         else:
#             return dataframe.groupby(by_column).agg(agg_dict).reset_index()
#     else:
#         if simple_agg_flag:
#             return dataframe.groupby(by_column).agg(agg_func)
#         else:
#             return dataframe.groupby(by_column).agg(agg_dict)
        
# def get_merge(dataframe1, dataframe2, on, axis=1, how='inner'):
#     return dataframe1.merge(dataframe2, on=on,how=how)

In [ ]:
transaction_base = get_groupby(transactions,'msno', agg_dict=None, agg_func = 'mean', simple_agg_flag=True, reset_index=True)
transaction_base.head()

In [ ]:
agg_dict = { 'transaction_date':'max', 'membership_expire_date':'max' }
transaction_date = get_groupby(transactions,'msno', agg_dict=agg_dict, agg_func = 'mean', simple_agg_flag=False, reset_index=True)
transaction_date.head()

In [ ]:
transaction_final = get_merge(transaction_base, transaction_date, on = 'msno') 
transaction_final.head()

In [ ]:
get_data_describe(transaction_final)

In [ ]:
%%time
get_data_profile(transaction_final,html_save_path=None, 
                     embed_in_cell=True,take_sample=False, sample_frac=0.01, 
                dataframe_name='Transaction_Final')

**Observations**:


#Write your observations from the profile report created above


#### User Behavioural Features

Like the transactions data, the data here is also present in one-many format.  Therefore this data also has to converted in one-one format.

In [ ]:
get_data_describe(user_logs)

In [ ]:
#Missing Values
get_data_na_values(user_logs)

In [ ]:
#this function is also available in utils.py 

# def get_fix_skew_with_log(dataframe, columns, replace_inf = True, replace_inf_with = 0):
#     if replace_inf:
#         dataframe_log = np.log(dataframe[columns]).replace([np.inf, -np.inf], replace_inf_with)
#         return pd.concat([dataframe_log, dataframe.drop(columns,axis=1)], axis=1)
#     else:
#         dataframe_log = np.log(dataframe[columns])
#         return pd.concat([dataframe_log, dataframe.drop(columns,axis=1)], axis=1)
        

In [ ]:
user_logs_transformed = get_fix_skew_with_log(user_logs, ['num_25','num_50','num_75','num_985','num_100','num_unq','total_secs'], 
                                              replace_inf = True, replace_inf_with = 0)
user_logs_transformed.head()

In [ ]:
get_data_describe(user_logs_transformed)

In [ ]:
user_logs_transformed.drop('date', axis=1, inplace= True)

In [ ]:
user_logs_transformed_final = get_groupby(user_logs_transformed,'msno', agg_dict=None, agg_func = 'mean', simple_agg_flag=True, reset_index=True)
user_logs_transformed_final.head()

In [ ]:
%%time
get_data_profile(user_logs_transformed_final,html_save_path=None, 
                     embed_in_cell=True,take_sample=False, sample_frac=0.01, 
                dataframe_name='user_logs_transformed_final')

**Observations**:


#Write your observations from the profile report created above


### 1.6 Joining the dataset

In [ ]:
print(members.shape)
print(train.shape)
print(transaction_final.shape)
print(user_logs_transformed_final.shape)

In [ ]:
%%time
train_df_v01 = get_merge(members, train, on='msno', axis=1, how='inner')
train_df_v02 = get_merge(train_df_v01, transaction_final, on='msno', axis=1, how='inner')
train_df_final = get_merge(train_df_v02, user_logs_transformed_final, on='msno', axis=1, how='inner')
train_df_final.head()

In [ ]:
# Rename columns using a dictionary
train_df_final = train_df_final.rename(columns={
    'transaction_date_x': 'transaction_date_min',
    'membership_expire_date_x': 'membership_expire_date_min',
    'transaction_date_y': 'transaction_date_max',
    'membership_expire_date_y': 'membership_expire_date_max'
})

In [ ]:
train_df_final.info()

In [ ]:
%%time
get_data_profile(train_df_final,html_save_path=None, 
                     embed_in_cell=True,take_sample=False, sample_frac=0.01, 
                dataframe_name='train_df_final')

**Observations**:

#Write your observations from the profile report created above


In [ ]:
#members_v01_sample_joinfinal.drop(['registration_init_time','date','transaction_date','membership_expire_date'],axis=1,inplace=True)

### 1.6.1 Saving the dataset

In [ ]:
#this function is also available in utils.py
# import time
# def get_save_intermediate_data(dataframe, path, filename="data_interim"):
#     filename = filename+"_"+str(int(time.time()))+".csv"
#     dataframe.to_csv(path+filename)
#     return "Data Saved Here :",path+filename

In [ ]:
%%time
get_save_intermediate_data(train_df_final, path=intermediate_data_path, filename="final_train_data_interim")